In [14]:
# TensorFlow Model !
import tensorflow as tf
from cell import ConvLSTMCell

# Run when your in trouble ... !
tf.reset_default_graph()

class conv_lstm_model():
    def __init__(self):
        """Parameter initialization"""
        self.batch_size = 128
        self.timesteps = 32
        self.shape = [64, 64] # Image shape
        self.kernel = [3, 3]
        self.channels = 3
        self.filters = [32,128,32,3] # 4 stacked conv lstm filters
        
        # Create a placeholder for videos.
        self.inputs = tf.placeholder(tf.float32, [batch_size, timesteps] + shape + [channels]) # (batch_size, timestep, H, W, C)
        self.outputs_exp = tf.placeholder(tf.float32, [batch_size, timesteps] + shape + [channels] ) # (batch_size, timestep, H, W, C)
        
        # model output
        self.model_output = None
        
        # loss
        self.l2_loss = None
        
        # optimizer
        self.optimizer = None
        
    def create_model(self):
        cells = []
        for i, each_filter in enumerate(self.filters):
            cell = ConvLSTMCell(self.shape, each_filter, self.kernel)
            cells.append(cell)
            
        cell = tf.nn.rnn_cell.MultiRNNCell(cells, state_is_tuple=True)        
        states_series, current_state = tf.nn.dynamic_rnn(cell, self.inputs, dtype=self.inputs.dtype)
        # current_state => Not used ... 
        self.model_output = states_series
    
    def loss(self):
        frames_difference = tf.subtract(self.outputs_exp, self.model_output)
        batch_l2_loss = tf.nn.l2_loss(frames_difference)
        # divide by batch size ... 
        l2_loss = tf.divide(batch_l2_loss, float(batch_size))
        self.l2_loss = l2_loss
    
    def optimize(self):
        train_step = tf.train.AdamOptimizer().minimize(self.l2_loss)
        self.optimizer = train_step
        
model = conv_lstm_model()
model.create_model()
model.loss()
model.optimize()

LSTMStateTuple(c=TensorShape([Dimension(64), Dimension(64), Dimension(12)]), h=TensorShape([Dimension(64), Dimension(64), Dimension(12)]))